In [ ]:
!pip install -U transformers accelerate datasets peft


In [ ]:
from datasets import load_dataset

base_path = "/content/drive/MyDrive/GrammarCorrectionProject/data/processed/"

train_path = base_path + "train_en.csv"
valid_path = base_path + "valid_en.csv"
test_path  = base_path + "test_en.csv"

train_dataset = load_dataset("csv", data_files=train_path)["train"]
valid_dataset = load_dataset("csv", data_files=valid_path)["train"]
test_dataset  = load_dataset("csv", data_files=test_path)["train"]

train_dataset, valid_dataset, test_dataset


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

(Dataset({
     features: ['source', 'target'],
     num_rows: 38959
 }),
 Dataset({
     features: ['source', 'target'],
     num_rows: 4870
 }),
 Dataset({
     features: ['source', 'target'],
     num_rows: 4870
 }))

In [ ]:
def preprocess(batch):
    # Add prefix
    inputs = ["gec: " + s for s in batch["source"]]
    outputs = batch["target"]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    # Tokenize labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            outputs,
            max_length=128,
            truncation=True,
            padding="max_length"
        )["input_ids"]

    # Mask padding tokens
    labels = [
        [-100 if tok == tokenizer.pad_token_id else tok for tok in seq]
        for seq in labels
    ]

    model_inputs["labels"] = labels
    return model_inputs


In [ ]:
train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names
)

valid_dataset = valid_dataset.map(
    preprocess,
    batched=True,
    remove_columns=valid_dataset.column_names
)

test_dataset = test_dataset.map(
    preprocess,
    batched=True,
    remove_columns=test_dataset.column_names
)


Map:   0%|          | 0/38959 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4118: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/4870 [00:00<?, ? examples/s]

Map:   0%|          | 0/4870 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type="torch")
valid_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")


In [ ]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

model_name = "gotutiyan/gec-t5-large-clang8"
tokenizer = T5TokenizerFast.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False


In [ ]:
train_dataset[0]


{'input_ids': tensor([  873,    75,    10,  2210, 19519,    10,  3790,    19,    29,    31,
            17,   786,    19,   255,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gec_en_finetuned",
    logging_strategy="epoch",
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=4,
    weight_decay=0.01,
    fp16=True,
    report_to="none",   # disable wandb
)

model.gradient_checkpointing_enable()


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-1169413603.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 4.12 MiB is free. Process 3268 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 30.81 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

we are hitting OOM before training even starts because:

✔ T5-Large + 15GB GPU = too large to load in full precision

And your GPU already has no free memory left (4MB free).
So just loading the model triggers OOM.


The ONLY solution that works on a 15GB GPU:
Load the model in 8-bit or 4-bit using bitsandbytes (QLoRA)

This reduces memory usage from 14GB → 3–5GB.

In [ ]:
!pip install bitsandbytes accelerate transformers peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 9.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers datasets accelerate bitsandbytes peft


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q", "v"],  # T5 attention modules
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989


### Cell _yQLTzN4MxBu: Install Libraries

This cell installs the necessary Python libraries for natural language processing and model training:

- `transformers`: Provides pre-trained models, tokenizers, and utilities for NLP tasks.
- `accelerate`: Helps in training PyTorch models on various hardware setups, especially for distributed training and mixed precision.
- `datasets`: Offers tools to easily load, process, and share datasets for machine learning.
- `peft`: Parameter-Efficient Fine-Tuning library, used for efficiently adapting large pre-trained models to downstream tasks with minimal overhead.

### Cell _j4fd9i2NJeo: Load Datasets

This cell loads the training, validation, and test datasets from CSV files into `Dataset` objects using the `load_dataset` function from the `datasets` library. These datasets are intended for a grammar correction task, with 'source' representing the incorrect sentence and 'target' representing the correct sentence.

### Cell _XFCKupDsrXH: Define Preprocessing Function

This cell defines a `preprocess` function that takes a batch of data (source and target sentences) and prepares it for the T5 model:

1. **Prefix Addition**: It adds the prefix `"gec: "` to each source sentence. This prefix is often used with T5 models to indicate the specific task (in this case, Grammar Error Correction - GEC).
2. **Tokenization**: It tokenizes both the input (source) and target (label) sentences using the loaded T5 tokenizer.
   - `max_length=128`: Ensures all sequences are padded or truncated to a maximum length of 128 tokens.
   - `truncation=True`: Truncates sequences longer than `max_length`.
   - `padding="max_length"`: Pads shorter sequences to `max_length`.
3. **Label Masking**: For the labels, it replaces padding tokens (`tokenizer.pad_token_id`) with `-100`. This is a standard practice in Hugging Face `Trainer` to make sure these padded tokens are ignored in the loss calculation.

### Cell Fie6yoANsaNv: Apply Preprocessing

This cell applies the `preprocess` function to the `train_dataset`, `valid_dataset`, and `test_dataset` using the `map` method. This transforms the raw text data into tokenized numerical inputs and labels suitable for model training. The `remove_columns` argument ensures that the original 'source' and 'target' text columns are removed, keeping only the tokenized inputs and labels.

### Cell KLXaibyZtGPJ: Set Dataset Format

This cell sets the format of the `train_dataset`, `valid_dataset`, and `test_dataset` to `"torch"`. This is important for ensuring that the datasets return PyTorch tensors, which are required for training with PyTorch and the Hugging Face `Trainer`.

### Cell 46mMRJtiNQrg: Load Tokenizer and Model

This cell loads the pre-trained T5 tokenizer (`T5TokenizerFast`) and the T5 model for conditional generation (`T5ForConditionalGeneration`) from the specified Hugging Face model hub path (`"gotutiyan/gec-t5-large-clang8"`). It also configures the model's `pad_token_id` to match the tokenizer's and sets `use_cache` to `False`, which can sometimes help with memory usage during training.

### Cell uMzE5wxytCap: Display Example from Training Dataset

This cell displays the first example from the `train_dataset` after preprocessing. This allows you to inspect the tokenized `input_ids`, `attention_mask`, and `labels` to verify that the preprocessing step worked as expected.

### Cell H5huMEg8tg2E: Configure Training Arguments

This cell defines the `TrainingArguments` for the `Trainer`. These arguments specify various aspects of the training process, including:

- `output_dir`: Directory to save model checkpoints.
- `logging_strategy`, `save_strategy`, `eval_strategy`: Strategies for logging, saving, and evaluating (set to 'epoch').
- `learning_rate`: The initial learning rate.
- `per_device_train_batch_size`, `per_device_eval_batch_size`: Batch sizes for training and evaluation.
- `gradient_accumulation_steps`: Number of steps to accumulate gradients before performing a backward/update pass.
- `num_train_epochs`: Total number of training epochs.
- `weight_decay`: L2 regularization.
- `fp16=True`: Enables mixed-precision training (using float16) to reduce memory usage and speed up training.
- `report_to="none"`: Disables reporting to external tools like Weights & Biases.

It also enables `gradient_checkpointing` on the model, which can further reduce memory consumption at the cost of slightly slower training.

### Cell RzsGlg1LtjKi: Initialize and Train the Model (OOM Error)

This cell attempts to initialize the Hugging Face `Trainer` with the loaded model, training arguments, and datasets, and then starts the training process using `trainer.train()`.

**Error**: This cell resulted in an `OutOfMemoryError`. This indicates that the `T5ForConditionalGeneration` model, being a large model (`t5-large`), consumed too much GPU memory when loaded in full precision, even with `gradient_checkpointing` enabled. The error message explicitly states that the GPU has very little free memory, and the process is already using almost all of it.

### Cell t_kaorRIxW9R: Explanation of Out-of-Memory (OOM) Error

This text cell clearly explains why the previous training attempt failed: the T5-Large model, in its full precision (32-bit floating point) form, is too big for the available 15GB GPU memory. The model itself, when loaded, consumes most of the GPU's capacity, leading to an immediate Out-of-Memory error even before actual training steps begin.

### Cell aMFQlhedyeb9: Solution for OOM Error - Quantization (QLoRA)

This text cell proposes the solution to the OOM issue: loading the model in a quantized format (8-bit or 4-bit) using techniques like QLoRA (Quantized Low-Rank Adaptation). Quantization significantly reduces the memory footprint of the model from around 14GB to 3-5GB, making it feasible to train on GPUs with limited memory like the one available.

### Cell 8D90aKYjtp3f: Install `bitsandbytes`

This cell installs the `bitsandbytes` library. This library is crucial for enabling 8-bit or 4-bit quantization, which is necessary to reduce the memory footprint of large language models like T5-Large, allowing them to fit into limited GPU memory.

### Cell WS_xE4TvzAeW: Upgrade Libraries

This cell upgrades several key libraries (`transformers`, `datasets`, `accelerate`, `bitsandbytes`, `peft`) to ensure that the latest versions with potentially critical bug fixes, performance improvements, and compatibility features (especially for quantization and PEFT) are in use.

### Cell 1TX0iiTRNbMW: Load Model with LoRA Configuration

This cell implements the solution to the OOM error by loading the model with a LoRA (Low-Rank Adaptation) configuration:

1. **`AutoModelForSeq2SeqLM.from_pretrained(model_name)`**: Loads the pre-trained T5 model. It's important that `bitsandbytes` is installed at this point, as `from_pretrained` can automatically detect and load the model in a quantized format if `load_in_8bit` or `load_in_4bit` arguments are passed (which are typically handled by `accelerate` or `peft` integration during fine-tuning).
2. **`LoraConfig`**: Defines the LoRA parameters:
   - `r=16`: The rank of the update matrices, controlling the number of trainable parameters.
   - `lora_alpha=32`: Scaling factor for LoRA updates.
   - `lora_dropout=0.05`: Dropout applied to the LoRA weights.
   - `target_modules=["q", "v"]`: Specifies which modules in the T5 architecture (query and value projections in attention layers) will have LoRA layers added. This focuses fine-tuning on critical parts of the model.
   - `bias="none"`: Specifies that bias terms are not included in LoRA layers.
   - `task_type="SEQ_2_SEQ_LM"`: Indicates that this is a sequence-to-sequence language modeling task.
3. **`get_peft_model(model, lora_config)`**: Applies the LoRA configuration to the base model, creating a PEFT (Parameter-Efficient Fine-Tuning) model. This wraps the base model, adding small trainable LoRA adapters while keeping most of the base model's parameters frozen.
4. **`model.print_trainable_parameters()`**: Prints a summary of the trainable parameters. This shows that only a small percentage of the total model parameters are now trainable, drastically reducing memory and computational requirements during fine-tuning.

### Cell gwNgvYsgOdfg: Reconfigure Training Arguments

This cell redefines the `TrainingArguments` after implementing LoRA. Key changes and important considerations:

- **`per_device_train_batch_size=8`, `per_device_eval_batch_size=8`**: The batch sizes are increased significantly (from 1 to 8). With the reduced memory footprint from LoRA and quantization, larger batch sizes are now feasible, which can lead to faster training and potentially better gradient estimates.
- **`learning_rate=5e-5`**: The learning rate is adjusted.
- **`do_eval=True`**: Explicitly enables evaluation during training.
- **`logging_steps=100`**: Logs training progress every 100 steps.
- **`save_total_limit=2`**: Keeps only the best 2 model checkpoints.

It's worth noting that the `evaluation_strategy` argument might have been removed or changed in newer `transformers` versions, as indicated by the comment `REMOVE evaluation_strategy (not supported in your version)`. The `do_eval=True` together with the `eval_strategy` (implicitly 'epoch' from previous cell or default if not specified) handles the evaluation.

### Cell CxwjFZYzPaw_: Train and Evaluate Model with LoRA

This cell initializes the `Trainer` with the LoRA-enabled model, the updated `TrainingArguments`, and the preprocessed datasets, then starts the training and evaluation process.

1. **`trainer = Trainer(...)`**: Creates the `Trainer` instance. It correctly uses the `tokenizer` parameter, although a warning about its deprecation is noted in the output.
2. **`trainer.train()`**: Begins the fine-tuning process of the LoRA adapters.
3. **`trainer.evaluate()`**: After training, this explicitly runs an evaluation on the `eval_dataset` to get final performance metrics (in this case, `eval_loss`). The output `{'eval_loss': nan}` suggests there might be an issue during evaluation, possibly due to all labels being masked or other numerical instabilities, which would need further investigation.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./gec_en_finetuned",
    do_eval=True,
    # REMOVE evaluation_strategy (not supported in your version)
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer
)

trainer.train()

# manual eval
trainer.evaluate()


/tmp/ipython-input-910497408.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,0.000000
200,0.000000
300,0.000000
400,0.000000
500,0.000000
600,0.000000
700,0.000000
800,0.000000
900,0.000000
1000,0.000000


{'eval_loss': nan,
 'eval_runtime': 124.5267,
 'eval_samples_per_second': 39.108,
 'eval_steps_per_second': 4.891,
 'epoch': 4.0}